In [1]:
import pandas as pd
import string
transcripts_val = pd.read_pickle("/data2/brain2text/b2t_25/transcripts_val.pkl")
transcripts_val_cleaned = pd.read_pickle("/data2/brain2text/b2t_25/transcripts_val_cleaned.pkl")



In [23]:
import string
import re

# 1. Extract the words
cased_words = set()

for item in transcripts_val:
    sentence = item[0]
    words = sentence.split()
    
    # We look at words from index 1 onwards to avoid the sentence-start capital
    # We also explicitly look for "I" anywhere, because it's a common failure mode
    if len(words) > 1:
        for word in words[1:]:
            # Clean punctuation (e.g., "Paris." -> "Paris")
            clean_word = word.strip(string.punctuation)
            
            if not clean_word: continue

            # Condition: It is capitalized in ground truth AND it is not just a digit
            if clean_word[0].isupper() and not clean_word.isdigit():
                cased_words.add(clean_word)

# Add "I" manually just in case it only appeared at start of sentences in your val set
cased_words.add("I") 

print(f"Found {len(cased_words)} words to capitalize.")
# print(cased_words) # Uncomment to inspect

Found 121 words to capitalize.


In [27]:
!python -m spacy download en_core_web_trf

/home/ebrahim/brainaudio/.venv/bin/python: No module named pip


In [ ]:
import spacy

# 1. LOAD THE TRANSFORMER (This is the critical fix)
try:
    nlp = spacy.load("en_core_web_trf")
except:
    print("Transformer not found, falling back to LG (results will be worse)")
    nlp = spacy.load("en_core_web_lg")

def smart_capitalize(text_lowercase):
    doc = nlp(text_lowercase)
    
    corrected_words = []
    for token in doc:
        # A. Always capitalize "I"
        if token.text == "i":
            corrected_words.append("I")
            
        # B. Check for Proper Nouns (POS) OR Named Entities (NER)
        # trf is smart enough to tag 'bill' as PERSON even if lowercase
        elif token.pos_ == "PROPN" or token.ent_type_ != "":
            corrected_words.append(token.text.capitalize())
            
        # C. First word of sentence
        elif token.i == 0:
            corrected_words.append(token.text.capitalize())
            
        else:
            corrected_words.append(token.text)
            
        corrected_words.append(token.whitespace_)

    return "".join(corrected_words)

# --- Test ---
text = "i went to paris with bill to buy a new cadillac."
print(smart_capitalize(text))

I went to Paris with bill to buy a new Cadillac.
